In [1]:
import pandas as pd
import re
import numpy as np
import ast


In [15]:
df = pd.read_csv("features_partial.csv")

# Danceability 
Essentia returns an aggregate danceability score and a framewise curve. For the initial project scope, extracting the aggregated number should be sufficient. These values usually range from 0 to 3 (higher values meaning more danceable).

In [16]:
df["track_danceability"] = df["Danceability"].apply(
    lambda x: round(float(re.search(r"\(([\d\.]+)", x).group(1)), 4)
)
print(df.head())

                                        Danceability         bpm  \
0  (1.0577861070632935, array([1.3124456 , 1.2694...  135.828598   
1  (1.0577861070632935, array([1.3124456 , 1.2694...  135.828598   
2  (1.1427139043807983, array([1.0092717 , 0.9785...  136.526443   
3  (1.0489819049835205, array([1.2052338 , 1.1632...  109.182640   
4  (0.9722141623497009, array([1.0625372 , 1.0406...   90.967430   

                                     rhythm_strength       key  \
0  [0.63854873 0.63854873 0.6269387  0.522449   0...  0.058114   
1  [0.63854873 0.63854873 0.6269387  0.522449   0...  0.058114   
2  [0.7894784  0.7778685  0.80108833 0.7662585  0...  0.083988   
3  [0.487619   0.46439916 0.45278907 0.476009   0...  0.055757   
4  [0.5688889  0.5572789  0.5572789  0.5688889  0...  0.051498   

                                        scale_vector       track  \
0  [31.765575    9.234179   29.007633    1.034228...     track_0   
1  [31.765575    9.234179   29.007633    1.034228...     t

# BPM
The beats per minute (BPM) score is a fairly straight forward Tempo measure. There's little need to transform at this stage of the project. 

# Rythm Strength
This is currently captured as a per-frame measurement of rythmic strength. To enable swift comparison, I am going to create two variables; one for the mean and one for the variance, i.e. does it chance much.

In [17]:
df["rhythm_strength"] = df["rhythm_strength"].apply(
    lambda x: np.fromstring(x.strip("[]"), sep=" ")
)

In [5]:
# calc mean per row
df["mean_rhythm_strength"] = df["rhythm_strength"].apply(
    lambda arr: np.mean(arr)
)

# calc variance per row
df["var_rhythm_strength"] = df["rhythm_strength"].apply(
    lambda arr: np.var(arr)
)

df.head()


,Danceability,bpm,rhythm_strength,key,scale_vector,track,youtube_url,track_danceability,mean_rhythm_strength,var_rhythm_strength
0,"(1.4968905448913574, array([1.0434638 , 1.0139...",123.938477,"[0.534059, 0.5340589, 0.54566884, 0.510839, 0....",0.083312,[ 8.903932 6.5607915 41.18719 7.289768...,track_861,https://youtube.com/watch?v=jQEcQNtouAg,1.4969,0.484415,0.000059
1,"(1.1244194507598877, array([0.86909485, 0.8541...",123.789642,"[0.46439907, 0.4643991, 0.4643991, 0.47600913,...",0.069519,[18.305223 18.366928 3.475936 2.920608...,track_863,https://youtube.com/watch?v=qNs-R9NU82I,1.1244,0.478455,0.002231
2,"(1.5002362728118896, array([0.89553595, 0.8321...",108.004150,"[0.5688889, 0.5572789, 0.5572789, 0.54566884, ...",0.070080,[37.781227 6.3873806 7.266615 0.310318...,track_864,https://youtube.com/watch?v=UrrRqk7HRlw,1.5002,0.555579,0.000024
3,"(1.1257785558700562, array([1.1045734 , 1.0890...",123.656113,"[0.49922907, 0.49922895, 0.47600913, 0.4992289...",0.061682,[28.751259 6.9234643 13.242699 1.636455...,track_866,https://youtube.com/watch?v=2NSR5rSk0Cs,1.1258,0.474866,0.000849
4,"(1.1556165218353271, array([0.9028398 , 0.8871...",128.019058,"[0.55727893, 0.54566896, 0.54566884, 0.5572789...",0.067957,[20.169893 4.681944 9.324378 5.788226...,track_872,https://youtube.com/watch?v=tky7vWXSZrs,1.1556,0.496172,0.002795


# Key 
It became apparent that I had unfortunately only saved the Confidence in the Key and Scale calculated by Essentia, but not the Key and Scale themeselves. The issue is that the pipeline I have built automatically removes the audio after analysis as audio files are large. Given the significant time it takes to download, I have asked Copilot the following "I no longer have the audio files to run Essentia. Can I use the PCP vector to calculate the Key and Scale?"

In [20]:
def normalize_profile(p):
    p = np.clip(p, 0, None)
    s = p.sum()
    return p / s if s > 0 else p

def fold_to_12(p):
    if len(p) % 12 != 0:
        raise ValueError("Profile length must be a multiple of 12.")
    n = len(p) // 12
    return p.reshape(12, n).sum(axis=1)

# Krumhansl–Schmuckler templates
major_template = np.array([6.35,2.23,3.48,2.33,4.38,4.09,
                           2.52,5.19,2.39,3.66,2.29,2.88])
minor_template = np.array([6.33,2.68,3.52,5.38,2.60,3.53,
                           2.54,4.75,3.98,2.69,3.34,3.17])
major_template /= major_template.sum()
minor_template /= minor_template.sum()

pitch_names = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]

def best_key(pcp12, template):
    scores = []
    for shift in range(12):
        rotated = np.roll(template, shift)
        num = np.dot(pcp12, rotated)
        den = np.linalg.norm(pcp12) * np.linalg.norm(rotated)
        scores.append(num/den if den > 0 else 0.0)
    best_shift = int(np.argmax(scores))
    return best_shift, scores[best_shift]

def estimate_key_from_profile(profile):
    p = np.array(profile, dtype=float)
    p12 = normalize_profile(fold_to_12(normalize_profile(p)))
    maj_shift, maj_score = best_key(p12, major_template)
    min_shift, min_score = best_key(p12, minor_template)
    if maj_score >= min_score:
        return f"{pitch_names[maj_shift]} major"
    else:
        return f"{pitch_names[min_shift]} minor"

df["key_scale"] = df["scale_vector"].apply(
    lambda x: estimate_key_from_profile(np.fromstring(x.strip("[]"), sep=" "))
)

df.head()

,Danceability,bpm,rhythm_strength,key,scale_vector,track,youtube_url,track_danceability,key_scale
0,"(1.0577861070632935, array([1.3124456 , 1.2694...",135.828598,"[0.63854873, 0.63854873, 0.6269387, 0.522449, ...",0.058114,[31.765575 9.234179 29.007633 1.034228...,track_0,https://youtube.com/watch?v=u45UQVGRVPA,1.0578,C minor
1,"(1.0577861070632935, array([1.3124456 , 1.2694...",135.828598,"[0.63854873, 0.63854873, 0.6269387, 0.522449, ...",0.058114,[31.765575 9.234179 29.007633 1.034228...,track_1,https://youtube.com/watch?v=u45UQVGRVPA,1.0578,C minor
2,"(1.1427139043807983, array([1.0092717 , 0.9785...",136.526443,"[0.7894784, 0.7778685, 0.80108833, 0.7662585, ...",0.083988,[ 5.126527 3.5994763 11.1474695 2.748691...,track_10,https://youtube.com/watch?v=Pv7GJkqtNuc,1.1427,G# major
3,"(1.0489819049835205, array([1.2052338 , 1.1632...",109.182640,"[0.487619, 0.46439916, 0.45278907, 0.476009, 0...",0.055757,[41.443398 2.172339 15.954622 0.265508...,track_100,https://youtube.com/watch?v=lWINVYSsrTU,1.0490,C major
4,"(0.9722141623497009, array([1.0625372 , 1.0406...",90.967430,"[0.5688889, 0.5572789, 0.5572789, 0.5688889, 0...",0.051498,[22.47191 7.794944 14.349251 4.002809...,track_1002,https://youtube.com/watch?v=SXlIN3mcsH4,0.9722,C minor


# Pitch Class Profile (PCP) (scale_vector)
Summarizes tonal information across a track (regardless of octave). For example. Dance tracks are likely to have a clear tonal centre. 

In [21]:
df["scale_vector"] = df["scale_vector"].apply(
    lambda x: np.fromstring(x.strip("[]"), sep=" ")
)

In [22]:
df["var_PCP"]   = df["scale_vector"].apply(np.var)
df["max_PCP"]   = df["scale_vector"].apply(np.max)
df["dominant_pitch"] = df["scale_vector"].apply(lambda arr: np.argmax(arr))

pitch_classes = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]

df["dominant_pitch_class"] = df["dominant_pitch"].apply(
    lambda idx: pitch_classes[idx % 12] )

df.tail(20)

,Danceability,bpm,rhythm_strength,key,scale_vector,track,youtube_url,track_danceability,key_scale,var_PCP,max_PCP,dominant_pitch,dominant_pitch_class
307,"(1.00360906124115, array([1.0515424 , 1.035172...",166.419998,"[0.55727893, 0.5572789, 0.557279, 0.5572789, 0...",0.047485,"[18.538595, 21.335646, 30.659149, 4.5750217, 7...",track_768,https://youtube.com/watch?v=azw4Kh8Rqpw,1.0036,C# major,59.950183,30.659149,2,D
308,"(1.210000991821289, array([0.9396195 , 0.93717...",133.836166,"[0.44117913, 0.46439904, 0.47600913, 0.4643989...",0.066387,"[28.232971, 11.920039, 8.859822, 6.194472, 12....",track_777,https://youtube.com/watch?v=fZ5B6w-Baxs,1.2100,C major,39.979335,28.232971,0,C
309,"(1.0260013341903687, array([1.1282192 , 1.0812...",101.644493,"[0.499229, 0.522449, 0.5340589, 0.54566884, 0....",0.059789,"[14.161782, 8.393904, 22.907385, 5.345838, 11....",track_779,https://youtube.com/watch?v=g0AQI_Nt1X4,1.0260,A# minor,29.373031,22.907385,2,D
310,"(1.160965085029602, array([1.0300467 , 1.00730...",126.011253,"[0.47600907, 0.47600907, 0.476009, 0.47600913,...",0.072219,"[48.444912, 1.686874, 14.681075, 2.6093833, 2....",track_785,https://youtube.com/watch?v=0nusBQ7Lacg,1.1610,C major,101.269850,48.444912,0,C
311,"(1.3071630001068115, array([0.9774067 , 0.9507...",132.003861,"[0.4527891, 0.47600907, 0.47600913, 0.4643991,...",0.062013,"[10.326933, 8.588479, 33.990658, 18.033213, 2....",track_807,https://youtube.com/watch?v=0c_2T-DZUe0,1.3072,C# major,57.076866,33.990658,2,D
312,"(1.2390305995941162, array([1.0249194 , 1.0248...",172.265472,"[0.35990933, 0.34829926, 0.34829938, 0.3599093...",0.039581,"[47.727272, 4.5199184, 3.2686415, 8.146068, 1....",track_910,https://youtube.com/watch?v=RRQlsvWMWBo,1.2390,C major,110.963165,47.727272,0,C
313,"(0.9982732534408569, array([1.0360991 , 1.0149...",100.828697,"[0.3599093, 0.35990924, 0.3599093, 0.3599093, ...",0.059770,"[4.6327415, 5.68891, 4.1286607, 1.4402305, 1.5...",track_919,https://youtube.com/watch?v=z7OvpjplJ_8,0.9983,A minor,27.275074,22.203552,19,G
314,"(1.087671160697937, array([1.0670943 , 1.01523...",172.265213,"[0.5572789, 0.5688889, 0.557279, 0.5572789, 0....",0.065264,"[9.90778, 28.32821, 24.946796, 3.9016316, 0.63...",track_924,https://youtube.com/watch?v=10XR67NQcAc,1.0877,C major,52.818909,28.328210,1,C#
315,"(0.9926857948303223, array([1.2901363 , 1.2610...",114.033974,"[0.5688889, 0.5688889, 0.55727875, 0.5688889, ...",0.071031,"[16.750597, 2.4118738, 6.679035, 2.0408163, 3....",track_93,https://youtube.com/watch?v=gn8GMilGDZw,0.9927,C major,12.885366,16.750597,0,C
316,"(1.1648662090301514, array([1.0372434 , 0.9856...",112.914597,"[0.60371876, 0.6037189, 0.5921087, 0.58049893,...",0.072397,"[23.898306, 4.6731234, 6.0290556, 8.74092, 0.4...",track_930,https://youtube.com/watch?v=asZwDUTEXls,1.1649,E minor,43.964658,23.898306,0,C


In [23]:
df.to_csv("audiofeatures.csv", index=False)


# now join to table on youtube url